# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo_client = MongoClient('mongodb://localhost:27017/')

In [3]:
# confirm that our new database was created
dbs = mongo_client.list_database_names()
if "uk_food" in dbs:
    print("Database 'uk_food' found.")
else:
    print("Database 'uk_food' NOT found.")

Database 'uk_food' found.


In [5]:
# assign the uk_food database to a variable name
db = mongo_client['uk_food']

In [7]:
# review the collections in our new database
collections = db.list_collection_names()
if "establishments" in collections:
    print("Collection 'establishments' found.")
else:
    print("Collection 'establishments' NOT found.")

Collection 'establishments' found.


In [8]:
# assign the collection to a variable
establishments = db['establishments']
sample_doc = establishments.find_one()
pprint(sample_doc)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6761f026be54c58d768eb0c9'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)
print("New restaurant 'Penang Flavours' added successfully.")

New restaurant 'Penang Flavours' added successfully.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = result["BusinessTypeID"]  # Replace this if you stored it manually

establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {"$set": {"BusinessTypeID": business_type_id}}
)

print("Penang Flavours updated with the correct BusinessTypeID.")

Penang Flavours updated with the correct BusinessTypeID.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [13]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": {"$regex": "Dover", "$options": "i"}})
print(f"Number of documents in Dover: {dover_count}")

Number of documents in Dover: 994


In [14]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": {"$regex": "Dover", "$options": "i"}})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [15]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents({"LocalAuthorityName": {"$regex": "Dover", "$options": "i"}})
print(f"Number of documents in Dover after deletion: {dover_count_after}")

Number of documents in Dover after deletion: 0


In [16]:
# Check that other documents remain with 'find_one'
remaining_doc = establishments.find_one()
print("Sample remaining document after deletion:")
pprint(remaining_doc)

Sample remaining document after deletion:
{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6761f026be54c58d768eb3ad'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [17]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {"$set": {
            "geocode.latitude": {"$toDouble": "$geocode.latitude"},
            "geocode.longitude": {"$toDouble": "$geocode.longitude"}
        }}
    ]
)
print("Latitude and longitude fields converted to decimal numbers.")

Latitude and longitude fields converted to decimal numbers.


Use `update_many` to convert `RatingValue` to integer numbers.

In [18]:
# Set non 1-5 Rating Values to Null
establishments.update_many(
    {},
    [
        {"$set": {
            "RatingValue": {
                "$cond": {
                    "if": {"$regexMatch": {"input": "$RatingValue", "regex": "^[0-9]+$"}},
                    "then": {"$toInt": "$RatingValue"},
                    "else": None
                }
            }
        }}
    ]
)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# Check that the coordinates and rating value are now numbers
print("RatingValue fields converted to integers where applicable.")

RatingValue fields converted to integers where applicable.
